In [71]:
from src.linear_causality import LinearLKInformationFlow
import numpy as np
from tqdm import tqdm
# from causality_subspace import causality_subspace

dt=1
pn=1
T1=[];T2=[]
# pca = PCA(n_components=5)  # 假设我们想将数据降到2维  
Nt=1000000
np.random.seed = 5
# for i in tqdm(range(100)):
X1=np.random.randn(5,Nt)*0.2
X2=np.random.randn(5,Nt)*0.2

A1=np.array([[0.9,0.6,0,0.6,0],[0,0.8,0.6,0,0.6],[0,0,0.9,0,0],[0,0,0,0.2,0],[0,0,0,0,0.2]]).T
A2=np.array([[0.2,0.6,0,0.6,0],[0,0.2,0.6,0,0.6],[0,0,0.2,0,0],[0,0,0,0.2,0],[0,0,0,0,0.2]]).T

B1=-np.array([[0,0,0,0,0],[0,0.,0,0,0],[0,0,0,0.9,0.],[0,0,0,0.,0],[0,0,0,0,0]])
B2=-np.array([[0,0,0,0,0],[0,0.0,0,0,0],[0,0,0,0,0],[0,0,0,0.4,0],[0,0,0,0,0]])
for it in range(1,Nt):
    X1[:,it]=A1@X1[:,it-1]+X1[:,it]+1*B1@X2[:,it-1] 
    X2[:,it]=A2@X2[:,it-1]+X2[:,it]+1*B2@X1[:,it-1] 
    
XX=np.zeros([10,Nt])
XX[:5]=X1
XX[5:]=X2

In [78]:
segments = [(2,3),(8,9)]
segments = [(0, 5),(5,10)]

In [112]:
lkif = LinearLKInformationFlow(np,1)
lkif.causality_estimate(XX[:,-15000:].T, lag_list=[1], segments=segments)

In [113]:
result_dict = lkif.get_dict()

In [114]:
np.abs(result_dict["information_flow"]) > result_dict["information_flow_std_origin"]*2.56

array([[ True,  True],
       [ True,  True]])

In [115]:
np.abs(result_dict["information_flow"]) > result_dict["statistics"]["p99_critical_value"]

array([[ True,  True],
       [ True,  True]])

In [111]:
np.abs(result_dict["information_flow"])

array([[2.00481158e+00, 3.38060437e-03],
       [2.47244279e-01, 4.00879888e+00]])

In [105]:
result_dict["information_flow_std_origin"]

array([[0.01538825, 0.0014133 ],
       [0.00431541, 0.01945412]])

In [106]:
result_dict["information_flow_std"]

array([[0.01538825, 0.00157665],
       [0.00631235, 0.01945412]])

---

In [116]:
## load data/X.dat and Y.dat
import pandas as pd
data_X = pd.read_csv('data/X.dat', header=None)
data_Y = pd.read_csv('data/Y.dat', header=None)

In [117]:
data_X = data_X.to_numpy()
data_Y = data_Y.to_numpy()

In [118]:
data = np.concatenate((data_X, data_Y), axis=1)

In [119]:
segments = [(0,1),(1,2)]

In [128]:
lkif = LinearLKInformationFlow(np,1)
lkif.causality_estimate(data, lag_list=[1], segments=segments)

In [129]:
result_dict = lkif.get_dict()

In [143]:
print('## 是否存在因果关系 (False 代表无信息流)\n')
print("### 使用原本的方差检验方法")
print("```")
print(np.abs(result_dict["information_flow"]) > result_dict["information_flow_std_origin"] * 1.65)
print("```")

print("### 使用新的方差检验方法")
print("```")
print(np.abs(result_dict["information_flow"]) > result_dict["information_flow_std"] * 1.65)
print("```")


## 是否存在因果关系 (False 代表无信息流)

### 使用原本的方差检验方法
```
[[ True False]
 [ True  True]]
```
### 使用新的方差检验方法
```
[[ True False]
 [False  True]]
```


---

In [1]:
## bootstrap
from src.linear_causality import LinearLKInformationFlow
import numpy as np
from tqdm import tqdm
# from causality_subspace import causality_subspace

dt=1
pn=1
T1=[];T2=[]
# pca = PCA(n_components=5)  # 假设我们想将数据降到2维  
Nt=100000
np.random.seed = 5
# for i in tqdm(range(100)):
X1=np.random.randn(3,Nt)*0.2
X2=np.random.randn(3,Nt)*0.2

A1=np.array([[0.9,0.6,0,],[0,0.8,0.6,],[0,0,0.9,]]).T
A2=np.array([[0.2,0.6,0,],[0,0.2,0.6,],[0,0,0.2]]).T

B1=-np.array([[0,0,0],[0,0.,0],[0,0,0]])
B2=-np.array([[0,0,0],[0,0.,0],[0,0,0]])
for it in range(1,Nt):
    X1[:,it]=A1@X1[:,it-1]+X1[:,it]+1*B1@X2[:,it-1] 
    X2[:,it]=A2@X2[:,it-1]+X2[:,it]+1*B2@X1[:,it-1] 
    
XX=np.zeros([6,Nt])
XX[:3]=X1
XX[3:]=X2

In [2]:
# bootstrap
import numpy as np
from sklearn.utils import resample
from tqdm import tqdm


In [3]:
segments = [(0,1),(1,2),(2,3),(3,4),(4,5),(5,6)]
lkif = LinearLKInformationFlow(np,1)
lkif.causality_estimate(XX[:,-15000:].T, lag_list=[1], segments=segments)
result_dict = lkif.get_dict()

In [9]:
lkif.bootstrap_estimate(XX[:,-15000:].T, lag_list=[1], segments=segments, bootstrap_num=1000)

(array([[-9.53703731e-02, -8.16821360e-05,  4.07374100e-05,
          1.44900042e-06,  1.72674061e-04, -2.29192830e-05],
        [ 1.75063005e-01, -1.98402093e-01, -1.58748860e-03,
          8.57125322e-06,  1.71034826e-05,  6.27050334e-07],
        [ 4.41374715e-06,  9.64044827e-02, -1.00072247e-01,
         -5.54259995e-07, -6.90266377e-07, -2.59863932e-06],
        [ 1.92903599e-05,  7.36130130e-05,  2.57818166e-04,
         -7.85155166e-01,  1.69712024e-04,  1.59447481e-04],
        [ 1.43233369e-04,  3.43376975e-05,  3.60515245e-05,
          6.56002443e-02, -7.96963883e-01,  1.30710035e-03],
        [ 9.89579222e-05,  4.79705966e-05, -2.17306098e-05,
          5.45872773e-05,  9.56078571e-02, -8.01184591e-01]]),
 array([[5.50347279e-03, 5.80939946e-03, 2.61065504e-03, 4.18498054e-05,
         1.12800912e-04, 5.57831262e-05],
        [2.03645118e-03, 3.02298171e-03, 1.72774450e-03, 1.66940335e-05,
         2.47855786e-05, 1.24779169e-05],
        [2.03353904e-04, 8.24596486e-04, 4

In [10]:
result_dict["information_flow_std_origin"]

array([[5.60428326e-03, 5.93978802e-03, 2.64374540e-03, 3.98989368e-06,
        5.60506302e-05, 4.68660674e-05],
       [1.62106592e-03, 2.99256233e-03, 1.69838161e-03, 8.10346010e-06,
        1.06290412e-05, 3.56937466e-06],
       [2.05339430e-04, 4.83346159e-04, 4.59714552e-04, 4.74441997e-06,
        1.81043380e-06, 3.53808025e-06],
       [1.45774359e-05, 1.08482725e-04, 2.23177403e-04, 8.04340517e-03,
        1.05133852e-03, 2.01979102e-04],
       [1.69381048e-04, 1.17692470e-04, 7.04391721e-05, 8.69575063e-04,
        6.84873642e-03, 1.16132227e-03],
       [1.35819756e-04, 3.79024113e-05, 1.32013817e-04, 1.60210529e-04,
        1.11371193e-03, 6.46412690e-03]])

In [11]:
result_dict["information_flow_std"]

array([[5.60428326e-03, 5.93978819e-03, 2.64374573e-03, 2.82159560e-05,
        1.07339595e-04, 4.93280140e-05],
       [2.03596785e-03, 2.99256233e-03, 1.69841369e-03, 1.27577076e-05,
        2.21132507e-05, 4.34297415e-06],
       [2.05339513e-04, 8.07814982e-04, 4.59714552e-04, 4.74861612e-06,
        2.02581545e-06, 3.87904195e-06],
       [1.69352687e-04, 1.43370569e-04, 2.53738827e-04, 8.04340517e-03,
        1.05138764e-03, 2.09006068e-04],
       [1.85008287e-04, 1.23054069e-04, 8.21017444e-05, 4.21944002e-03,
        6.84873642e-03, 1.16297388e-03],
       [1.47858243e-04, 1.36682487e-04, 1.32670457e-04, 1.61141736e-04,
        4.64846748e-03, 6.46412690e-03]])